In [1]:
import pandas as pd
from datetime import date, timedelta
import json

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
death_words = ['DEATH','DEAD','DIED']

def clean_outcome(row):
    if row['outcome'] in death_words:
        return 'DEATH'
    else:
        return 'POSITIVE'

In [ ]:
cnty_join = pd.read_csv('../data/tl_2019_us_county.csv',dtype={'STATEFP':str,'COUNTYFP':str,'GEOID':str})
cnty_join = cnty_join[['STATEFP','COUNTYFP','GEOID','NAMELSAD']]

state = 'California'
state_fips = '06'
today = str(date.today())
print("Today's date:", today)

case_data = pd.read_csv('../data/case-data/outside_Hubei.data.19032020T011105.csv')
us_cases = case_data.loc[case_data['country'] == 'United States']
state_cases = us_cases.loc[us_cases['province'] == state]
state_cases['date_confirm_clean'] = pd.to_datetime(state_cases['date_confirmation'], format='%d.%m.%Y', errors='ignore')

state_cases['outcome'] = state_cases['outcome'].str.upper()
state_cases['outcome_clean'] = state_cases.apply(lambda row: clean_outcome(row), axis=1)

#join county fips
cnty_join = cnty_join.loc[cnty_join['STATEFP'] == state_fips]
cnty_join = cnty_join[['NAMELSAD','COUNTYFP']]
cnty_join_dict = dict(zip(cnty_join.NAMELSAD, cnty_join.COUNTYFP))     
state_cases['fips'] = state_cases['city'].map(cnty_join_dict)

#pivot on county, agg deaths positive cases
by_county = pd.pivot_table(state_cases, values='ID', index=['province','fips','city','date_confirm_clean'],
                           columns=['outcome_clean'], aggfunc='count')

print('Total cases reported for',state,':',len(state_cases))
print('Total deaths reported for',state,':',len(state_cases.loc[state_cases['outcome_clean'] == 'DEATH']))
print('Most recent case confirmation date:',state_cases.date_confirm_clean.max())
display(by_county)

In [ ]:
state_cases.head()

## County + day dataset

In addition to the above cases per county per day, we need to record county + days that have no cases or deaths so we can get a good time series.

In [ ]:
covid_start = date(2020, 1, 21)   # start date
today = date.today()   # end date
delta = today - covid_start       # as timedelta

days = []
for i in range(delta.days + 1):
    day = covid_start + timedelta(days=i)
    days.append(day)
    
fullDateTime = pd.DataFrame(columns=['date','fips','county'])
for day in days:
    for index, row in cnty_join.iterrows():
        fullDateTime = fullDateTime.append({'date': day, 
                                            'fips': row['COUNTYFP'], 
                                            'county': row['NAMELSAD']}, ignore_index=True)
        
fullDateTime['date_confirm_clean'] = pd.to_datetime(fullDateTime['date'], format='%Y-%m-%d', errors='ignore')
fullDateTime['date_str'] = fullDateTime['date_confirm_clean'].dt.strftime('%Y-%m-%d')
fullDateTime['join_field'] = fullDateTime['date_str'] + '-' + fullDateTime['fips']

by_county_min = by_county.reset_index()
by_county_min['date_str'] = by_county_min['date_confirm_clean'].dt.strftime('%Y-%m-%d')
by_county_min['join_field'] = by_county_min['date_str'] + '-' + by_county_min['fips']

if 'DEATH' in by_county_min.columns:
    keep_cols = ['date', 'fips_x', 'county','POSITIVE','DEATH']
else:
    keep_cols = ['date', 'fips_x', 'county','POSITIVE']
    
fullDT_joined = fullDateTime.merge(by_county_min,how='left',on='join_field')
fullDT_joined = fullDT_joined.fillna(0)
fullDT_joined = fullDT_joined[keep_cols]

fullDT_joined.to_csv('../app/assets/data/'+str(today)+'-'+state+'-export.csv',index=False)

In [ ]:
fullDT_joined.loc[fullDT_joined['POSITIVE']>0]

In [ ]:
fullDT_joined.POSITIVE.sum()